In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("ExampleApp").getOrCreate()

In [3]:
departures_df = spark.read.csv('data/AA_DFW_2015_Departures_Short.csv.gz', header=True)

In [4]:
# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3] > 0)

# Add an ID column
departures_df = departures_df.withColumn('id', F.monotonically_increasing_id())

# Write the file out to JSON format
# departures_df.write.json('.cache/output.json', mode='overwrite')

In [5]:
# # Import the file to a DataFrame and perform a row count
# annotations_df = spark.read.csv('annotations.csv.gz', sep='|')
# full_count = annotations_df.count()

# # Count the number of rows beginning with '#'
# comment_count = annotations_df.where(col('_c0').startswith('#')).count()

# # Import the file to a new DataFrame, without commented rows
# no_comments_df = spark.read.csv('annotations.csv.gz', sep='|', comment='#')

# # Count the new DataFrame and verify the difference is as expected
# no_comments_count = no_comments_df.count()
# print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

In [6]:
# # Split _c0 on the tab character and store the list in a variable
# tmp_fields = F.split(annotations_df['_c0'], '\t')

# # Create the colcount column on the DataFrame
# annotations_df = annotations_df.withColumn('colcount', F.size(tmp_fields))

# # Remove any rows containing fewer than 5 fields
# annotations_df_filtered = annotations_df.filter(~ (annotations_df["colcount"] < 5))

# # Count the number of rows
# final_count = annotations_df_filtered.count()
# print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

In [7]:
# # Split the content of _c0 on the tab character (aka, '\t')
# split_cols = F.split(annotations_df["_c0"], '\t')

# # Add the columns folder, filename, width, and height
# split_df = annotations_df.withColumn('folder', split_cols.getItem(0))
# split_df = split_df.withColumn('filename', split_cols.getItem(1))
# split_df = split_df.withColumn('width', split_cols.getItem(2))
# split_df = split_df.withColumn('height', split_cols.getItem(3))

# # Add split_cols as a column
# split_df = split_df.withColumn('split_cols', split_cols)


In [8]:
# def retriever(cols, colcount):
#   # Return a list of dog data
#   return cols[4:colcount]

# # Define the method as a UDF
# udfRetriever = F.udf(retriever, ArrayType(StringType()))

# # Create a new column using your UDF
# split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))

# # Remove the original column, split_cols, and the colcount
# split_df = split_df.drop('_c0').drop('split_cols').drop('colcount')

## Data Validation

In [9]:
# # Rename the column in valid_folders_df
# valid_folders_df = valid_folders_df.withColumnRenamed('_c0', 'folder')

# # Count the number of rows in split_df
# split_count = split_df.count()

# # Join the DataFrames
# joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")

# # Compare the number of rows remaining
# joined_count = joined_df.count()
# print("Before: %d\nAfter: %d" % (split_count, joined_count))

In [10]:
# # Determine the row counts for each DataFrame
# split_count = split_df.count()
# joined_count = joined_df.count()

# # Create a DataFrame containing the invalid rows
# invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')

# # Validate the count of the new DataFrame is as expected
# invalid_count = invalid_df.count()
# print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

# # Determine the number of distinct folder rows removed
# invalid_folder_count = invalid_df.select('folder').distinct().count()
# print("%d distinct invalid folders found" % invalid_folder_count)

## Final Analysis and Delivery

In [11]:
# # Select the dog details and show 10 untruncated rows
# print(joined_df.select('dog_list').show(10, truncate=False))

# # Define a schema type for the details in the dog list
# DogType = StructType([
# 	StructField("breed", StringType(), False),
#     StructField("start_x", IntegerType(), False),
#     StructField("start_y", IntegerType(), False),
#     StructField("end_x", IntegerType(), False),
#     StructField("end_y", IntegerType(), False)
# ])

In [12]:
# # Create a function to return the number and type of dogs as a tuple
# def dogParse(doglist):
#   dogs = []
#   for dog in doglist:
#     (breed, start_x, start_y, end_x, end_y) = dog.split(',')
#     dogs.append((breed, int(start_x), int(start_y), int(end_x), int(end_y)))
#   return dogs

# # Create a UDF
# udfDogParse = F.udf(dogParse, ArrayType(DogType))


# # Use the UDF to list of dogs and drop the old column
# joined_df = joined_df.withColumn('dogs', udfDogParse('dog_list')).drop('dog_list')


# # Show the number of dogs in the first 10 rows
# joined_df.select(F.size('dogs')).show(10)




In [13]:
# # Define a UDF to determine the number of pixels per image
# def dogPixelCount(doglist):
#   totalpixels = 0
#   for dog in doglist:
#     totalpixels += (dog[3] - dog[1]) * (dog[4] - dog[2])
#   return totalpixels

# # Define a UDF for the pixel count
# udfDogPixelCount = F.udf(dogPixelCount, IntegerType())
# joined_df = joined_df.withColumn('dog_pixels', udfDogPixelCount('dogs'))

# # Create a column representing the percentage of pixels
# joined_df = joined_df.withColumn('dog_percent', (joined_df.dog_pixels / (joined_df.width * joined_df.height)) * 100)

# # Show the first 10 annotations with more than 60% dog
# joined_df.where('dog_percent > 60').show(10)